# 数値積分 (異なる手法の比較)

## 台形公式

$$\int_a^b f(x) \mathrm{d}x$$の数値的近似解を求めたい。ただし、関数は連続で滑らか。
区間を等間隔な幅$h$を持つ区間に分割しする。
$$\int_a^b f(x) \mathrm{d}x = \sum_{n=1}^{N-1} \int_{a_n}^{a_{n+1}}f(x) \mathrm{d}x.$$
ただし、$$a_1=a, a_N=b.$$
ここで、各区間の積分を以下のように近似する。
$$\int_{a_n}^{a_{n+1}}f(x) \mathrm{d}x \simeq \frac{f(a_n) + f(a_{n+1})}{2} h.$$
すべての項をまとめると、
$$\int_a^b f(x) \mathrm{d}x \simeq h\sum_{n=1}^{N-1} \frac{f(a_n) + f(a_{n+1})}{2} = h \sum_{n=1}^N f(a_{n}) - h f(a_1)/2 - h f(a_N)/2.$$
$$f(x) = f(x_0) + f(x_0)^\prime + O(x)$$
結果の誤差は、分割数$N$に対して、$O(1/N^2)$。

## シンプソンの公式

積分区間中で非積分関数を多項式で近似する。
まず、ラグランジュ補間を導入する。
任意の関数$f(x)$に対して、異なる点$x_0,~x_1,\cdots,x_n$での値$f(x_0),~f(x_1),\cdots,f(x_n)$が得られているとする。
ラグランジュ補間は、$n+1$個の多項式の和として、
$$
p_n(x) = \sum_{j=0}^n f(x_j) l_j(x)
$$
と書ける。ここで、
$$
l_j(x) = \prod_{k\neq j} \frac{x-x_k}{x_j-x_k}
$$
である。$l_j(x)$は$n$次多項式であり、
$$
 l_j(x_k)=
 \begin{cases}
    0 &  k \neq j\\
    1 &  k = j.\\
  \end{cases}
$$
この性質を使うと、$p_n(x_k) = f(x_k)$であることが分かる。
つまり、$p_n(x)$は、関数$f(x)$の$n$次多項式での補間である。
なお、$f(x)$が$n$次以下の多項式の場合には、補間は厳密である [$f(x)=p_n(x)~\forall x$]。
多くの数値積分公式は、非積分関数を多項式近似した後、その補間された関数を厳密に積分することで得られる。
特に、$n=2$の場合には、ラグランジュ補間を解析的に積分することで、以下の積分公式が得られる。
$$
\int_a^b f(x) \mathrm{d} x = \frac{b-a}{6} \left[f(a) + 4 f\left(\frac{a+b}{2}\right) + f(b)\right].
$$
なお、この場合、非積分関数を補間した際の残差は$O(h^3)$である。
台形公式の場合には、$O(h^2)$であった。
なので、今回の場合には、数値積分自体の誤差は、$O(h^3)$であることが期待される。


In [53]:
import numpy as np

p = 4

def f(x):
    return x**p

b = 1.0
a = 0
intf = ((b-a)/6) * (f(a) + 4*f((a+b)/2) + f(b))
print(intf, b**(p+1)/(p+1) - a**(p+1)/(p+1))

0.20833333333333331 0.2


## 合成則

$$
\int_a^b f(x) \mathrm{d} x = \sum_{i=0}^{N-1} \int_{a_i}^{a_{i+1}} f(x) \mathrm{d} x =  \sum_{i=0}^{N-1}   \frac{a_{i+1}-a_i}{6} \left[f(a_i) + 4 f\left(\frac{a_i+a_{i+1}}{2}\right) + f(a_{i+1})\right].
$$

In [54]:
def simpson(f, a, b):
    return ((b-a)/6) * (f(a) + 4*f((a+b)/2) + f(b))

print(simpson(f, 0, 1))

def composite_simpson(f, a, b, N):
    xn = np.linspace(a, b, N+1)
    r = 0.0
    for i in range(N):
        r += simpson(f, xn[i], xn[i+1])
    return r

#def g(x):
#    return np.sqrt(1-x**2)

a = 0.0
b = 1.0
for N in [1, 10, 100, 1000, 1000000]:
    print(N, composite_simpson(f, a, b, N)- b**(p+1)/(p+1) + a**(p+1)/(p+1))


0.20833333333333331
1 0.00833333333333
10 8.33333333339e-07
100 8.33333124728e-11
1000 8.49320613838e-15
1000000 -5.3290705182e-15


In [44]:
def g(x):
    return np.sqrt(1-x**2)

a =  0.0
b =  1.0
for N in [1, 10, 100, 1000, 3000]:
    print(N, 4*composite_simpson(g, a, b, N) - np.pi)
    
#あまり精度が出ない

1 -0.165524910165
10 -0.00514558933259
100 -0.000162404396773
1000 -5.13467751873e-06
3000 -9.8815487215e-07


In [43]:
for N in [1, 10, 100, 1000]:
    print(3*N, 4*(composite_simpson(g, 0, 0.9, N) + composite_simpson(g, 0.9, 0.99, N)+ composite_simpson(g, 0.99, 1.0, N)) - np.pi)

3 -0.0200720905577
30 -1.88804402015e-05
300 -1.63978103096e-07
3000 -5.13472819819e-09


In [50]:
def h(x):
    return np.sin(x)**2

for N in [1, 2]:
    print(N, 4*composite_simpson(h, 0, 0.5*np.pi, N) - np.pi)

1 -8.881784197e-16
2 -4.4408920985e-16
